# Финальный проект. Промышленность. 

# Дополнительное задание 1

Выполните задание

Вам поручили изучить характеристики процесса обработки стали. Для каждого значения столбца key найдите:

    Длительность времени между первым и последним замером температуры.
    Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.
    Количество запусков нагрева электродами.
    Среднее соотношение потребления активной и реактивной мощности.

По всем полученным столбцам вычислите статистики: средние, минимальные и максимальные значения, медиану и величины 25%- и 75%-квартилей.

In [1]:
import pandas as pd

In [2]:
data_arc = pd.read_csv('data_arc.csv')
data_temp = pd.read_csv('data_temp.csv')

## Длительность времени между первым и последним замером температуры.

In [3]:
# переведем в формат даты
data_temp['Время замера'] = data_temp['Время замера'].astype('datetime64')

In [4]:
# Длительность времени между первым и последним замером температуры.

# сводная таблица с результатами измерения температуры
data_temp_by_key = data_temp.pivot_table(index='key', aggfunc={'Время замера': ('first', 'last')})

In [5]:
# избавляемся от мультииндекса и именуем столбцы
#data_temp_by_key.columns = data_temp_by_key.columns.droplevel()
data_temp_by_key.index.name = None
# data_temp_by_key.set_axis(['А/Р сред', 'кол-во нагревов', 'время нагревов сумм'], axis='columns', inplace=True)
data_temp_by_key.columns = ['время начальное', 'время конечное']

In [6]:
data_temp_by_key

,время начальное,время конечное
1,2019-05-03 11:16:18,2019-05-03 11:30:39
2,2019-05-03 11:37:27,2019-05-03 11:59:12
3,2019-05-03 12:13:17,2019-05-03 12:34:57
4,2019-05-03 12:52:57,2019-05-03 12:59:25
5,2019-05-03 13:23:19,2019-05-03 13:36:01
...,...,...
3237,2019-08-31 22:44:04,2019-08-31 23:05:43
3238,2019-08-31 23:30:31,2019-08-31 23:43:00
3239,2019-09-01 01:31:47,2019-09-01 02:24:15
3240,2019-09-01 02:39:01,2019-09-01 03:35:16


In [7]:
data_temp_by_key['начало-конец измерений'] = (
    data_temp_by_key['время конечное'] - data_temp_by_key['время начальное']).astype('int64')/1000000000

In [8]:
data_temp_by_key['начало-конец измерений']

1        861.0
2       1305.0
3       1300.0
4        388.0
5        762.0
         ...  
3237    1299.0
3238     749.0
3239    3148.0
3240    3375.0
3241    2078.0
Name: начало-конец измерений, Length: 3216, dtype: float64

In [9]:
data_temp_by_key['начало-конец измерений'].describe()

count     3216.000000
mean      1839.861629
std       1246.950298
min          0.000000
25%       1177.000000
50%       1581.000000
75%       2235.000000
max      23537.000000
Name: начало-конец измерений, dtype: float64

## Суммарное время нагрева электродами.
* Cумму значений по всем промежуткам между запусками нагрева электродов.
* Количество запусков нагрева электродами.
* Среднее соотношение потребления активной и реактивной мощности.

In [10]:
# переведем в формат даты
data_arc['Начало нагрева дугой'] = data_arc['Начало нагрева дугой'].astype('datetime64')
data_arc['Конец нагрева дугой'] = data_arc['Конец нагрева дугой'].astype('datetime64')

In [11]:
# для каждого нагрева посчитаем длительность нагрева и соотношение потребления активной и реактивной мощности
data_arc['Длительность нагрева']=(data_arc['Конец нагрева дугой']-data_arc['Начало нагрева дугой']).astype('int64')/1000000000
data_arc['А/Р'] = data_arc['Активная мощность']/data_arc['Реактивная мощность']

In [12]:
data_arc_by_key = data_arc.pivot_table( index='key', aggfunc={ 'Длительность нагрева': ('sum', 'count'), 'А/Р': 'mean'})

In [13]:
# избавляемся от мультииндекса и именуем столбцы
#data_arc_by_key.columns = data_arc_by_key.columns.droplevel()
data_arc_by_key.index.name = None
#data_arc_by_key.set_axis(['А/Р сред', 'кол-во нагревов', 'время нагревов сумм'], axis='columns', inplace=True)
data_arc_by_key.columns = ['А/Р сред', 'кол-во нагревов', 'время нагревов сумм']
data_arc_by_key

,А/Р сред,кол-во нагревов,время нагревов сумм
1,1.538898,5,1098.0
2,1.551834,4,811.0
3,1.583120,5,655.0
4,1.563487,4,741.0
5,1.480428,4,869.0
...,...,...,...
3237,1.384203,5,909.0
3238,1.257794,3,546.0
3239,1.497918,8,1216.0
3240,1.257343,5,839.0


In [14]:
data_arc_by_key.describe()

,А/Р сред,кол-во нагревов,время нагревов сумм
count,3214.000000,3214.000000,3214.000000
mean,1.357452,4.628500,794.579029
std,0.132824,1.608647,332.448622
min,0.676106,1.000000,57.000000
25%,1.292892,4.000000,571.000000
50%,1.370550,4.000000,770.000000
75%,1.438292,6.000000,983.000000
max,1.782164,16.000000,4189.000000
